In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

from functools import reduce
from itertools import combinations

from scipy import stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# configure pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
dpath = '/home/cglab/projects/sui/data/'

### load suicide data

In [3]:
sui = pd.read_csv(dpath + 'abcd5.1_sui_wide_count.csv')
print(sui.shape)
sui.head()

(11878, 168)


,ksads_23_143_t_1,ksads_23_144_t_1,ksads_23_145_t_1,ksads_23_146_t_1,ksads_23_147_t_1,ksads_23_148_t_1,ksads_23_149_t_1,ksads_23_150_t_1,ksads_24_151_t_1,ksads_24_152_t_1,ksads_24_153_t_1,ksads_24_154_t_1,ksads_23_807_t_1,ksads_23_808_t_1,ksads_23_809_t_1,ksads_23_810_t_1,ksads_23_811_t_1,ksads_23_812_t_1,ksads_23_813_t_1,ksads_23_814_t_1,ksads_23_815_t_1,ksads_23_816_t_1,ksads_23_817_t_1,ksads_23_818_t_1,ksads_23_819_t_1,ksads_23_820_t_1,ksads_23_821_t_1,ksads_23_822_t_1,ksads_23_823_t_1,ksads_23_824_t_1,ksads_23_825_t_1,ksads_23_945_t_1,ksads_23_946_t_1,ksads_23_947_t_1,ksads_23_948_t_1,ksads_23_949_t_1,ksads_23_950_t_1,ksads_23_951_t_1,ksads_23_952_t_1,ksads_23_953_t_1,ksads_23_954_t_1,ksads_23_955_t_1,ksads_23_956_t_1,ksads_23_957_t_1,ksads_23_958_t_1,ksads_23_959_t_1,ksads_23_960_t_1,ksads_23_961_t_1,ksads_23_962_t_1,ksads_23_963_t_1,ksads_23_964_t_1,ksads_23_965_t_1,ksads_23_966_t_1,ksads_23_143_t_t3,ksads_23_144_t_t3,ksads_23_145_t_t3,ksads_23_146_t_t3,ksads_23_147_t_t3,ksads_23_148_t_t3,ksads_23_149_t_t3,ksads_23_150_t_t3,ksads_24_151_t_t3,ksads_24_152_t_t3,ksads_24_153_t_t3,ksads_24_154_t_t3,ksads_23_807_t_t3,ksads_23_808_t_t3,ksads_23_809_t_t3,ksads_23_810_t_t3,ksads_23_811_t_t3,ksads_23_812_t_t3,ksads_23_813_t_t3,ksads_23_814_t_t3,ksads_23_815_t_t3,ksads_23_816_t_t3,ksads_23_817_t_t3,ksads_23_818_t_t3,ksads_23_819_t_t3,ksads_23_820_t_t3,ksads_23_821_t_t3,ksads_23_822_t_t3,ksads_23_823_t_t3,ksads_23_824_t_t3,ksads_23_825_t_t3,ksads_23_945_t_t3,ksads_23_946_t_t3,ksads_23_947_t_t3,ksads_23_948_t_t3,ksads_23_949_t_t3,ksads_23_950_t_t3,ksads_23_951_t_t3,ksads_23_952_t_t3,ksads_23_953_t_t3,ksads_23_954_t_t3,ksads_23_955_t_t3,ksads_23_956_t_t3,ksads_23_957_t_t3,ksads_23_958_t_t3,ksads_23_959_t_t3,ksads_23_960_t_t3,ksads_23_961_t_t3,ksads_23_962_t_t3,ksads_23_963_t_t3,ksads_23_964_t_t3,ksads_23_965_t_t3,ksads_23_966_t_t3,ksads_23_143_t_t5,ksads_23_144_t_t5,ksads_23_145_t_t5,ksads_23_146_t_t5,ksads_23_147_t_t5,ksads_23_148_t_t5,ksads_23_149_t_t5,ksads_23_150_t_t5,ksads_24_151_t_t5,ksads_24_152_t_t5,ksads_24_153_t_t5,ksads_24_154_t_t5,ksads_23_807_t_t5,ksads_23_808_t_t5,ksads_23_809_t_t5,ksads_23_810_t_t5,ksads_23_811_t_t5,ksads_23_812_t_t5,ksads_23_813_t_t5,ksads_23_814_t_t5,ksads_23_815_t_t5,ksads_23_816_t_t5,ksads_23_817_t_t5,ksads_23_818_t_t5,ksads_23_819_t_t5,ksads_23_820_t_t5,ksads_23_821_t_t5,ksads_23_822_t_t5,ksads_23_823_t_t5,ksads_23_824_t_t5,ksads_23_825_t_t5,ksads_23_945_t_t5,ksads_23_946_t_t5,ksads_23_947_t_t5,ksads_23_948_t_t5,ksads_23_949_t_t5,ksads_23_950_t_t5,ksads_23_951_t_t5,ksads_23_952_t_t5,ksads_23_953_t_t5,ksads_23_954_t_t5,ksads_23_955_t_t5,ksads_23_956_t_t5,ksads_23_957_t_t5,ksads_23_958_t_t5,ksads_23_959_t_t5,ksads_23_960_t_t5,ksads_23_961_t_t5,ksads_23_962_t_t5,ksads_23_963_t_t5,ksads_23_964_t_t5,ksads_23_965_t_t5,ksads_23_966_t_t5,subID,SA_1,SA_3,SA_5,SA_3_5,SI_1,SI_3,SI_5,SI_3_5
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NDAR_INV003RTV85,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN

#### Remove Suicide individual items
* keeping sumations

In [4]:
sitems = [c for c in sui if c.startswith('ksads')]
sui = sui.drop(columns=sitems)
print(sui.shape)

(11878, 9)


In [5]:
sui.columns

Index(['subID', 'SA_1', 'SA_3', 'SA_5', 'SA_3_5', 'SI_1', 'SI_3', 'SI_5', 'SI_3_5'], dtype='object')

In [6]:
sa_si = [c for c in sui.columns if c.startswith('SA') or c.startswith('SI')]
sa_si

['SA_1', 'SA_3', 'SA_5', 'SA_3_5', 'SI_1', 'SI_3', 'SI_5', 'SI_3_5']

In [7]:
sui.describe()

,SA_1,SA_3,SA_5,SA_3_5,SI_1,SI_3,SI_5,SI_3_5
count,11878.000000,11878.000000,11878.000000,11878.000000,11878.000000,11878.000000,11878.000000,11878.000000
mean,0.013134,0.011955,0.014060,0.022058,0.086294,0.076023,0.071140,0.123337
std,0.113851,0.108687,0.117742,0.146877,0.280809,0.265046,0.257069,0.328838
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
sui[sa_si].sum()

SA_1       156
SA_3       142
SA_5       167
SA_3_5     262
SI_1      1025
SI_3       903
SI_5       845
SI_3_5    1465
dtype: int64

### load MR MID, hses, and demo data

In [9]:
fpath = '/home/cglab/data_pull/abcd/'
df = pd.read_csv(fpath + 'abcd5.1_rtmri_mid_llosVn_lrwdVn_opfc_subc_net_hses_wide.csv')
print(df.shape)
df.head()

(9156, 134)


,NAL_lln1,NAR_lln1,AmygL_lln1,AmygR_lln1,HipcL_lln1,HipcR_lln1,ThalL_lln1,ThalR_lln1,subID,eventname1,NAL_lrn1,NAR_lrn1,AmygL_lrn1,AmygR_lrn1,HipcL_lrn1,HipcR_lrn1,ThalL_lrn1,ThalR_lrn1,LtOrFrL_lln1,LtOrFrR_lln1,MedOrFrL_lln1,MedOrFrR_lln1,rACCL_lln1,rACCR_lln1,cACCL_lln1,cACCR_lln1,cACCL_lrn1,cACCR_lrn1,LtOrFrL_lrn1,LtOrFrR_lrn1,MedOrFrL_lrn1,MedOrFrR_lrn1,rACCL_lrn1,rACCR_lrn1,aInslL_lln1,aInslR_lln1,aInslL_lrn1,aInslR_lrn1,famID1,age1,income1,pedu1,spedu1,gender1,race1,prpensity1,LowEdu11,SingPH11,UnempR11,tfmri_mid_all_meanmotion1,imgincl_mid_include1,scanID1,NAL_lln5,NAR_lln5,AmygL_lln5,AmygR_lln5,HipcL_lln5,HipcR_lln5,ThalL_lln5,ThalR_lln5,eventname5,NAL_lrn5,NAR_lrn5,AmygL_lrn5,AmygR_lrn5,HipcL_lrn5,HipcR_lrn5,ThalL_lrn5,ThalR_lrn5,LtOrFrL_lln5,LtOrFrR_lln5,MedOrFrL_lln5,MedOrFrR_lln5,rACCL_lln5,rACCR_lln5,cACCL_lln5,cACCR_lln5,cACCL_lrn5,cACCR_lrn5,LtOrFrL_lrn5,LtOrFrR_lrn5,MedOrFrL_lrn5,MedOrFrR_lrn5,rACCL_lrn5,rACCR_lrn5,aInslL_lln5,aInslR_lln5,aInslL_lrn5,aInslR_lrn5,age5,tfmri_mid_all_meanmotion5,imgincl_mid_include5,scanID5,NAL_lln9,NAR_lln9,AmygL_lln9,AmygR_lln9,HipcL_lln9,HipcR_lln9,ThalL_lln9,ThalR_lln9,eventname9,NAL_lrn9,NAR_lrn9,AmygL_lrn9,AmygR_lrn9,HipcL_lrn9,HipcR_lrn9,ThalL_lrn9,ThalR_lrn9,LtOrFrL_lln9,LtOrFrR_lln9,MedOrFrL_lln9,MedOrFrR_lln9,rACCL_lln9,rACCR_lln9,cACCL_lln9,cACCR_lln9,cACCL_lrn9,cACCR_lrn9,LtOrFrL_lrn9,LtOrFrR_lrn9,MedOrFrL_lrn9,MedOrFrR_lrn9,rACCL_lrn9,rACCR_lrn9,aInslL_lln9,aInslR_lln9,aInslL_lrn9,aInslR_lrn9,age9,tfmri_mid_all_meanmotion9,imgincl_mid_include9,scanID9
0,0.012925,0.057350,-0.103007,0.040944,-0.278594,-0.136520,-0.226935,-0.314408,NDAR_INV003RTV85,baseline_year_1_arm_1,-0.451426,-0.238997,-0.327643,-0.223388,-0.471753,-0.240092,-0.138320,-0.261594,-0.071183,-0.014068,0.184538,0.029904,0.053321,0.144795,-0.192958,0.058817,-0.089997,-0.015292,-0.320943,-0.315482,-0.250562,-0.804303,-0.311717,-0.233716,-0.224738,-0.105134,-0.274432,-0.184748,8781.0,131.0,8.0,13.0,13.0,2.0,1.0,466.092707,NaN,NaN,NaN,0.158333,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.429318,0.272447,0.354449,0.593416,0.234241,0.289909,0.017087,0.140608,NDAR_INV005V6D2C,baseline_year_1_arm_1,0.142344,0.284221,0.240394,0.450072,0.131028,0.231011,0.043190,0.068502,0.441208,0.173635,0.620966,0.564494,0.540422,0.546842,0.132752,0.293214,-0.080159,0.038292,0.384632,0.078511,0.239707,0.132052,0.406179,0.246562,0.212062,0.198078,0.052116,-0.058588,10210.0,121.0,999.0,6.0,999.0,2.0,3.0,520.488325,4.861931,29.079160,9.991899,0.352267,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.007370,-0.099530,-0.245714,-0.077979,-0.154156,-0.118741,-0.094763,-0.067210,NDAR_INV007W6H7B,baseline_year_1_arm_1,0.024064,-0.018880,-0.231686,-0.010857,-0.064591,-0.160605,-0.088715,-0.187258,-0.216364,-0.361254,0.003707,-0.123096,-0.181924,-0.007465,-0.092264,-0.091089,-0.186848,-0.159265,-0.259863,-0.227784,-0.275979,-0.273947,-0.375694,-0.290470,-0.060283,-0.222956,-0.010167,-0.118855,4722.0,126.0,10.0,19.0,18.0,1.0,1.0,479.185338,3.559711,0.000000,6.254295,0.086835,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.177538,-0.140450,-0.357152,-0.321334,-0.084301,-0.276284

In [10]:
df.isnull().sum()

NAL_lln1                       11
NAR_lln1                       11
AmygL_lln1                     11
AmygR_lln1                     11
HipcL_lln1                     11
HipcR_lln1                     11
ThalL_lln1                     11
ThalR_lln1                     11
subID                           0
eventname1                      0
NAL_lrn1                        9
NAR_lrn1                        9
AmygL_lrn1                      9
AmygR_lrn1                      9
HipcL_lrn1                      9
HipcR_lrn1                      9
ThalL_lrn1                      9
ThalR_lrn1                      9
LtOrFrL_lln1                   11
LtOrFrR_lln1                   11
MedOrFrL_lln1                  11
MedOrFrR_lln1                  11
rACCL_lln1                     11
rACCR_lln1                     11
cACCL_lln1                     11
cACCR_lln1                     11
cACCL_lrn1                      9
cACCR_lrn1                      9
LtOrFrL_lrn1                    9
LtOrFrR_lrn1  

### Merge Sui with overall df

In [11]:
df = df.merge(sui, how='left', on='subID')
df.shape

(9156, 142)

In [12]:
df.head()

,NAL_lln1,NAR_lln1,AmygL_lln1,AmygR_lln1,HipcL_lln1,HipcR_lln1,ThalL_lln1,ThalR_lln1,subID,eventname1,NAL_lrn1,NAR_lrn1,AmygL_lrn1,AmygR_lrn1,HipcL_lrn1,HipcR_lrn1,ThalL_lrn1,ThalR_lrn1,LtOrFrL_lln1,LtOrFrR_lln1,MedOrFrL_lln1,MedOrFrR_lln1,rACCL_lln1,rACCR_lln1,cACCL_lln1,cACCR_lln1,cACCL_lrn1,cACCR_lrn1,LtOrFrL_lrn1,LtOrFrR_lrn1,MedOrFrL_lrn1,MedOrFrR_lrn1,rACCL_lrn1,rACCR_lrn1,aInslL_lln1,aInslR_lln1,aInslL_lrn1,aInslR_lrn1,famID1,age1,income1,pedu1,spedu1,gender1,race1,prpensity1,LowEdu11,SingPH11,UnempR11,tfmri_mid_all_meanmotion1,imgincl_mid_include1,scanID1,NAL_lln5,NAR_lln5,AmygL_lln5,AmygR_lln5,HipcL_lln5,HipcR_lln5,ThalL_lln5,ThalR_lln5,eventname5,NAL_lrn5,NAR_lrn5,AmygL_lrn5,AmygR_lrn5,HipcL_lrn5,HipcR_lrn5,ThalL_lrn5,ThalR_lrn5,LtOrFrL_lln5,LtOrFrR_lln5,MedOrFrL_lln5,MedOrFrR_lln5,rACCL_lln5,rACCR_lln5,cACCL_lln5,cACCR_lln5,cACCL_lrn5,cACCR_lrn5,LtOrFrL_lrn5,LtOrFrR_lrn5,MedOrFrL_lrn5,MedOrFrR_lrn5,rACCL_lrn5,rACCR_lrn5,aInslL_lln5,aInslR_lln5,aInslL_lrn5,aInslR_lrn5,age5,tfmri_mid_all_meanmotion5,imgincl_mid_include5,scanID5,NAL_lln9,NAR_lln9,AmygL_lln9,AmygR_lln9,HipcL_lln9,HipcR_lln9,ThalL_lln9,ThalR_lln9,eventname9,NAL_lrn9,NAR_lrn9,AmygL_lrn9,AmygR_lrn9,HipcL_lrn9,HipcR_lrn9,ThalL_lrn9,ThalR_lrn9,LtOrFrL_lln9,LtOrFrR_lln9,MedOrFrL_lln9,MedOrFrR_lln9,rACCL_lln9,rACCR_lln9,cACCL_lln9,cACCR_lln9,cACCL_lrn9,cACCR_lrn9,LtOrFrL_lrn9,LtOrFrR_lrn9,MedOrFrL_lrn9,MedOrFrR_lrn9,rACCL_lrn9,rACCR_lrn9,aInslL_lln9,aInslR_lln9,aInslL_lrn9,aInslR_lrn9,age9,tfmri_mid_all_meanmotion9,imgincl_mid_include9,scanID9,SA_1,SA_3,SA_5,SA_3_5,SI_1,SI_3,SI_5,SI_3_5
0,0.012925,0.057350,-0.103007,0.040944,-0.278594,-0.136520,-0.226935,-0.314408,NDAR_INV003RTV85,baseline_year_1_arm_1,-0.451426,-0.238997,-0.327643,-0.223388,-0.471753,-0.240092,-0.138320,-0.261594,-0.071183,-0.014068,0.184538,0.029904,0.053321,0.144795,-0.192958,0.058817,-0.089997,-0.015292,-0.320943,-0.315482,-0.250562,-0.804303,-0.311717,-0.233716,-0.224738,-0.105134,-0.274432,-0.184748,8781.0,131.0,8.0,13.0,13.0,2.0,1.0,466.092707,NaN,NaN,NaN,0.158333,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
1,0.429318,0.272447,0.354449,0.593416,0.234241,0.289909,0.017087,0.140608,NDAR_INV005V6D2C,baseline_year_1_arm_1,0.142344,0.284221,0.240394,0.450072,0.131028,0.231011,0.043190,0.068502,0.441208,0.173635,0.620966,0.564494,0.540422,0.546842,0.132752,0.293214,-0.080159,0.038292,0.384632,0.078511,0.239707,0.132052,0.406179,0.246562,0.212062,0.198078,0.052116,-0.058588,10210.0,121.0,999.0,6.0,999.0,2.0,3.0,520.488325,4.861931,29.079160,9.991899,0.352267,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
2,-0.007370,-0.099530,-0.245714,-0.077979,-0.154156,-0.118741,-0.094763,-0.067210,NDAR_INV007W6H7B,baseline_year_1_arm_1,0.024064,-0.018880,-0.231686,-0.010857,-0.064591,-0.160605,-0.088715,-0.187258,-0.216364,-0.361254,0.003707,-0.123096,-0.181924,-0.007465,-0.092264,-0.091089,-0.186848,-0.159265,-0.259863,-0.227784,-0.275979,-0.273947,-0.375694,-0.290470,-0.060283,-0.222956,-0.010167,-0.118855,4722.0,126.0,10.0,19.0,18.0,1.0,1.0,479.185338,3.559711,0.000000,6.254295,0.086835,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### Export to csv

In [13]:
df.to_csv(fpath + 'abcd5.1_tmri_mid_llosVn_lrwdVn_opfc_subc_net_hses_wide_sui.csv', index=False)

# END